In [1]:
!pip install mlflow pyvi minio -q
!pip install hf_xet -q

import mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 59.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 96.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 100.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.2/700.2 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
mlflow.set_tracking_uri("http://36.50.135.226:7893/")

# from mlflow.tracking import MlflowClient

# client = MlflowClient()
# client.restore_experiment(experiment_id="2")

mlflow.set_experiment(experiment_id="10")

<Experiment: artifact_location='s3://mlflow/6', creation_time=1745743711248, experiment_id='10', last_update_time=1745743711248, lifecycle_stage='active', name='ImageCaption_TPC37k_BartPho-ViT-GPT2_LoRALayerFT', tags={}>

In [3]:
import os

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://36.50.135.226:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"

# MinIO

In [4]:
from minio import Minio
from minio.error import S3Error
import glob

# Training

In [5]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import transformers
from transformers import VisionEncoderDecoderModel, ViTImageProcessor
import torch
from PIL import Image
import torch.nn as nn
import cv2
import torchvision
from tqdm.notebook import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import ViTForImageClassification, ViTImageProcessor
from collections import OrderedDict
from transformers import GPT2Config, GPT2LMHeadModel
import mlflow
from mlflow.models import infer_signature
import mlflow.pytorch
import re
from pyvi import ViTokenizer
from torch.optim import AdamW

2025-05-12 13:36:11.034135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747056971.443562      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747056971.548976      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 1. Chuẩn bị file thư viện

In [6]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
with open("requirements.txt") as f:
    pip_reqs = [line.strip() for line in f if line.strip()]

# 2. Load data

In [8]:
import pandas as pd

def load_data(data_path="/kaggle/input/traffic-pictures-captioning/augmented/captions_augmented.csv"):
    try:
        df = pd.read_csv(data_path)
        df = df[['original_url','local_path', 'search_query', 'short_caption']].rename(columns={
            'original_url': 'original_url',
            'local_path': 'url',
            'search_query': 'search_query',
            'short_caption': 'caption'
        })
        print(f"Đã tải CSV từ: {data_path} (Kích thước: {df.shape})")
        return df
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại {data_path}")
        raise
    except Exception as e:
        print(f"Lỗi khi đọc CSV: {e}")
        raise

# Load dữ liệu
df = load_data()


Đã tải CSV từ: /kaggle/input/traffic-pictures-captioning/augmented/captions_augmented.csv (Kích thước: (37056, 4))


In [9]:

# # Chia DataFrame thành 3 phần bằng nhau
# n = len(df) // 3  # Số hàng mỗi phần (làm tròn xuống)
# df_1 = df.iloc[:n]
# df_2 = df.iloc[n:2*n]
# df_3 = df.iloc[2*n:]

# # In thông tin để kiểm tra
# print(f"Kích thước df_1: {df_1.shape}")
# print(f"Kích thước df_2: {df_2.shape}")
# print(f"Kích thước df_3: {df_3.shape}")

# 3. Split data

In [10]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

# Hàm chia dữ liệu giữ nguyên
def split_data(df, stratify_col='search_query', test_size=0.1, val_size=0.1, random_state=42):
    unique_urls = df.drop_duplicates('original_url')

    sss_1 = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_val_idx, test_idx in sss_1.split(unique_urls, unique_urls[stratify_col]):
        train_val_urls = unique_urls.iloc[train_val_idx]['original_url']
        test_urls = unique_urls.iloc[test_idx]['original_url']

    df_train_val = df[df['original_url'].isin(train_val_urls)]
    df_test = df[df['original_url'].isin(test_urls)]

    unique_train_val_urls = df_train_val.drop_duplicates('original_url')
    val_ratio = val_size / (1 - test_size)
    sss_2 = StratifiedShuffleSplit(n_splits=1, test_size=val_ratio, random_state=random_state)
    for train_idx, val_idx in sss_2.split(unique_train_val_urls, unique_train_val_urls[stratify_col]):
        train_urls = unique_train_val_urls.iloc[train_idx]['original_url']
        val_urls = unique_train_val_urls.iloc[val_idx]['original_url']

    df_train = df_train_val[df_train_val['original_url'].isin(train_urls)]
    df_val = df_train_val[df_train_val['original_url'].isin(val_urls)]

    return df_train.reset_index(drop=True), df_val.reset_index(drop=True), df_test.reset_index(drop=True)

In [11]:
def load_split_save(df: pd.DataFrame): 
    # Split
    train_df, val_df, test_df = split_data(df, stratify_col='search_query')
    
    # Kiểm tra
    print("Train size:", len(train_df))
    print("Val size:", len(val_df))
    print("Test size:", len(test_df))
    print(train_df['search_query'].value_counts(normalize=True))
    print(val_df['search_query'].value_counts(normalize=True))
    print(test_df['search_query'].value_counts(normalize=True))

    # Save file
    # Reset index
    train_df = train_df.reset_index(drop=True)
    val_df = val_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)
    
    # Kiểm tra số lượng mẫu sau khi chia
    print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
    
    # Lưu các DataFrame vào JSON
    train_df.to_json("train.json", orient='records', indent=4, force_ascii=False)
    val_df.to_json("val.json", orient='records', indent=4, force_ascii=False)
    test_df.to_json("test.json", orient='records', indent=4, force_ascii=False)
    
    print("Đã lưu train.json, val.json, test.json")
    return train_df, val_df, test_df, df

train_df, val_df, test_df, df = load_split_save(df)

Train size: 29640
Val size: 3708
Test size: 3708
search_query
vỉa hè đường phố việt nam               0.031039
đường phố sau bão                       0.023212
người đợi xe bus tại trạm               0.023077
nắng nóng đường phố việt nam            0.022807
người đi bộ tại ngã tư                  0.022267
                                          ...   
chướng ngại vật trên đường đi bộ        0.004049
công trình đào đường                    0.003914
vạch dành cho người khiếm thị           0.002834
biển báo chữ nổi cho người khiếm thị    0.001754
lối đi dành cho người khuyết tật        0.001350
Name: proportion, Length: 65, dtype: float64
search_query
vỉa hè đường phố việt nam               0.031284
đường phố sau bão                       0.023732
nắng nóng đường phố việt nam            0.022654
người đợi xe bus tại trạm               0.022654
người đi bộ tại ngã tư                  0.022654
                                          ...   
vạch kẻ đường cho người đi bộ           0.00431

# 4. Load feature extractor

In [12]:
feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

def load_image(local_path, dataset_base_path="/kaggle/input/traffic-pictures-captioning/"):
    """
    Load ảnh từ local_path trong dataset trên Kaggle.
    
    Parameters:
    - local_path (str): Đường dẫn tương đối của ảnh (từ cột 'url' trong DataFrame)
    - dataset_base_path (str): Đường dẫn gốc đến dataset
    
    Returns:
    - image_rgb (numpy.ndarray): Ảnh ở định dạng RGB, hoặc None nếu lỗi
    """
    try:
        # Chuẩn hóa đường dẫn ảnh
        local_path = local_path.lstrip('./')  # Loại bỏ './' nếu có
        full_image_path = os.path.join(dataset_base_path, local_path)
        
        # Đọc ảnh bằng OpenCV
        image = cv2.imread(full_image_path)
        if image is None:
            print(f"Lỗi: Không thể đọc ảnh từ {full_image_path}")
            return None
        
        # Chuyển từ BGR sang RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image_rgb
    
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh {full_image_path}: {e}")
        return None

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

# 5. Load Tokenizer

In [13]:
from transformers import AutoTokenizer

# Load tokenizer của BartPho
# tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")

# Kiểm tra vocab size
vocab_size = tokenizer.vocab_size
print(f"VOCAB SIZE: {vocab_size}")

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

VOCAB SIZE: 64000


In [14]:
# In ra các special tokens
print("Special Tokens:", tokenizer.special_tokens_map)

Special Tokens: {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}


# 6. Các hàm tiền xử lý caption

In [15]:
def clean_text(text: str) -> str:
    return re.sub(r"[^\w\s,!?.]", "", text).strip()

def to_lowercase(text: str) -> str:
    return text.lower()

def join_vietnamese_compounds(text: str) -> str:
    return ViTokenizer.tokenize(text)

def caption_preprocess(text: str) -> str:
    text = clean_text(text)
    text = to_lowercase(text)
    text = join_vietnamese_compounds(text)
    return text

In [16]:
text = caption_preprocess("Kiểm tra phân tách từ")

# Tách token
tokens = tokenizer.tokenize(text)

# Chuyển token thành ID
token_ids = tokenizer.convert_tokens_to_ids(tokens)

text = tokenizer.decode(token_ids, skip_special_tokens=True)
# In kết quả
print("List Word (Tokenized):", tokens)
print("List Token ID:", token_ids)
print("Text:", text)

List Word (Tokenized): ['kiểm_tra', 'phân_tách', 'từ']
List Token ID: [342, 31166, 39]
Text: kiểm_tra phân_tách từ


In [17]:
# Hàm tính số từ của một caption
def count_words(caption):
    preprocessed_caption = caption_preprocess(caption)
    tokens = tokenizer.tokenize(preprocessed_caption)
    return len(tokens)

# Áp dụng hàm cho toàn bộ dataframe
df['word_count'] = df['caption'].apply(count_words)

# Lấy độ dài lớn nhất
max_length = df['word_count'].max()

# In ra
print(f"Độ dài caption dài nhất là: {max_length} từ")

Độ dài caption dài nhất là: 71 từ


# Thống kê tập dữ liệu

In [18]:
# Kiểm tra vocal size
print(f"VOCAB SIZE: {vocab_size}")

# Kiểm tra maxlength từng tập train_df, val_df, test_df
df['word_count'] = train_df['caption'].apply(count_words)
max_length = df['word_count'].max()
mean_length = round(df['word_count'].mean(), 2)
print(f"Train max_length: {max_length} từ")
print(f"Train mean_length: {mean_length} từ")


df['word_count'] = val_df['caption'].apply(count_words)
max_length = df['word_count'].max()
mean_length = round(df['word_count'].mean(), 2)
print(f"Train max_length: {max_length} từ")
print(f"Train mean_length: {mean_length} từ")

df['word_count'] = test_df['caption'].apply(count_words)
max_length = df['word_count'].max()
mean_length = round(df['word_count'].mean(), 2)
print(f"Train max_length: {max_length} từ")
print(f"Train mean_length: {mean_length} từ")

VOCAB SIZE: 64000
Train max_length: 71.0 từ
Train mean_length: 31.34 từ
Train max_length: 64.0 từ
Train mean_length: 31.12 từ
Train max_length: 58.0 từ
Train mean_length: 31.16 từ


In [19]:
print("Số caption unique trong df:", df['caption'].nunique())
print("Số caption unique trong train df:", train_df['caption'].nunique())
print("Số caption unique trong val df:", val_df['caption'].nunique())
print("Số caption unique trong test df:", test_df['caption'].nunique())

Số caption unique trong df: 9081
Số caption unique trong train df: 7266
Số caption unique trong val df: 922
Số caption unique trong test df: 920


In [20]:
from tqdm import tqdm

from tqdm import tqdm

def get_token_word_stats(df, tokenizer, caption_preprocess):
    """
    Tính tổng số và số lượng duy nhất của tokens và words trong cột 'caption'

    Args:
        df (pd.DataFrame): DataFrame chứa cột 'caption'.
        tokenizer: Tokenizer đã load.
        caption_preprocess (func): Hàm xử lý caption trước khi tokenize.

    Returns:
        dict: {
            'total_tokens': int,
            'unique_tokens': int,
            'total_words': int,
            'unique_words': int
        }
    """
    unique_tokens = set()
    total_tokens = 0

    unique_words = set()
    total_words = 0

    for caption in tqdm(df['caption']):
        # Word: tách theo khoảng trắng
        words = caption.split()
        unique_words.update(words)
        total_words += len(words)
        
        text = caption_preprocess(caption)

        # Token: dựa vào tokenizer
        tokens = tokenizer.tokenize(text)
        unique_tokens.update(tokens)
        total_tokens += len(tokens)


    return total_tokens, len(unique_tokens), total_words, len(unique_words)


total_tokens, unique_tokens, total_words, unique_words = get_token_word_stats(df, tokenizer, caption_preprocess)

print("Type    |    Total    |    Unique    ")
print(f"Token   |   {total_tokens}   |     {unique_tokens}")
print(f"Work    |   {total_words}   |     {unique_words}")

100%|██████████| 37056/37056 [00:22<00:00, 1647.33it/s]

Type    |    Total    |    Unique    
Token   |   1160004   |     1731
Work    |   1267648   |     2356


# 7. Tiền xử lý input model

In [21]:
def process_data(image_url, caption):
    try:
        img_array = load_image(image_url)
        if img_array is None:
            return None
        
        pixel_values = feature_extractor(img_array, return_tensors="pt")["pixel_values"].squeeze(0)
        caption = caption_preprocess(caption)
        tokenized_caption = tokenizer(caption, padding="max_length", max_length=max_length, truncation=True)
        
        return {
            "pixel_values": pixel_values,
            "input_ids": torch.tensor(tokenized_caption["input_ids"]),
            "attention_mask": torch.tensor(tokenized_caption["attention_mask"])
        }
    except Exception as e:
        print(f"Error processing data: {e}")
        return None

# 8. Tạo tập dữ liệu huấn luyện

In [22]:
class ImageCaptionDataset(Dataset):
    def __init__(self, image_paths, captions):
        self.image_paths = image_paths
        self.captions = captions

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        data = process_data(self.image_paths[idx], self.captions[idx])
        if data is None:
            return self.__getitem__((idx + 1) % len(self))
        return data

def custom_collate_fn(batch):
    batch = [item for item in batch if item is not None]
    return {
        "pixel_values": torch.stack([item["pixel_values"] for item in batch]),
        "input_ids": torch.stack([item["input_ids"] for item in batch]),
        "attention_mask": torch.stack([item["attention_mask"] for item in batch])
    }

train_dataset = ImageCaptionDataset(train_df["url"], train_df["caption"])
val_dataset = ImageCaptionDataset(val_df["url"], val_df["caption"])

# 9. Cấu hình LoRA

In [23]:
from peft import get_peft_model, LoraConfig, TaskType
from peft import PeftModel

# Cấu hình LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],  # Các lớp trong GPT2
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# 10. Định nghĩa mô hình

In [24]:
vit_model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
del vit_model.classifier
vit_model.vit.pooler = torch.nn.Sequential(OrderedDict([
    ('dense', torch.nn.Linear(in_features=768, out_features=768, bias=True)),
    ('activation', torch.nn.Tanh())
]))

config = GPT2Config.from_pretrained("gpt2")
config.add_cross_attention = True
config.vocab_size = vocab_size

gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2", config=config, ignore_mismatched_sizes=True)
gpt2_model.resize_token_embeddings(config.vocab_size)

# Áp dụng LoRA vào GPT2
gpt2_model = get_peft_model(gpt2_model, lora_config)
gpt2_model.print_trainable_parameters()  # kiểm tra số lượng tham số cần huấn luyện

model = VisionEncoderDecoderModel(encoder=vit_model.vit, decoder=gpt2_model)

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['transformer.h.0.crossattention.c_attn.bias', 'transformer.h.0.crossattention.c_attn.weight', 'transformer.h.0.crossattention.c_proj.bias', 'transformer.h.0.crossattention.c_proj.weight', 'transformer.h.0.crossattention.q_attn.bias', 'transformer.h.0.crossattention.q_attn.weight', 'transformer.h.0.ln_cross_attn.bias', 'transformer.h.0.ln_cross_attn.weight', 'transformer.h.1.crossattention.c_attn.bias', 'transformer.h.1.crossattention.c_attn.weight', 'transformer.h.1.crossattention.c_proj.bias', 'transformer.h.1.crossattention.c_proj.weight', 'transformer.h.1.crossattention.q_attn.bias', 'transformer.h.1.crossattention.q_attn.weight', 'transformer.h.1.ln_cross_attn.bias', 'transformer.h.1.ln_cross_attn.weight', 'transformer.h.10.crossattention.c_attn.bias', 'transformer.h.10.crossattention.c_attn.weight', 'transformer.h.10.crossattention.c_proj.bias', 'transformer.h.10.cros

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 1,179,648 || all params: 164,540,928 || trainable%: 0.7169


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [25]:
print(model)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (inte

# 11. Định nghĩa lớp mô hình

In [ ]:
class LoRACaptionWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        super().__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def load_context(self, context):
        try:
            # Đường dẫn đầy đủ đến các thư mục con trong artifact
            model_path = context.artifacts["model_dir"]
            tokenizer_path = os.path.join(model_path, "tokenizer")
            feature_extractor_path = os.path.join(model_path, "feature_extractor")

            
            # Load các thành phần với đường dẫn chính xác
            self.model = VisionEncoderDecoderModel.from_pretrained(model_path).to(self.device)
            self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
            self.feature_extractor = ViTImageProcessor.from_pretrained(feature_extractor_path)
            
            # Đặt model vào chế độ eval
            self.model.eval()
        except Exception as e:
            raise ValueError(f"Error loading model: {str(e)}")

    def predict(self, context, model_input):
        try:
            # Xử lý đầu vào
            if isinstance(model_input, dict):
                image_url = model_input["url"][0] if "url" in model_input else model_input["image_path"][0]
            else:
                image_url = model_input.iloc[0]["url"] if "url" in model_input.columns else model_input.iloc[0]["image_path"]
            
            image = self.load_image(image_url)
            if image is None:
                return [""]
                
            # Tiền xử lý ảnh và tạo caption
            pixel_values = self.feature_extractor(images=image, return_tensors="pt").pixel_values.to(self.device)
            
            output_ids = self.model.generate(
                pixel_values,
                max_length=45,
                min_length=20,
                num_beams=4,
                do_sample=True,
                temperature=0.8,
                top_k=20,
                top_p=0.9,
                no_repeat_ngram_size=3,
                repetition_penalty=2.0,
                early_stopping=True,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                decoder_start_token_id=self.tokenizer.bos_token_id
            )
            
            caption = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
            return [caption.replace("_", " ").strip()]
            
        except Exception as e:
            print(f"[ERROR] during prediction: {e}")
            return [""]
   
    def load_image(self, image_url):
        try:
            import requests
            import numpy as np
            from PIL import Image
            from io import BytesIO
            
            response = requests.get(image_url, timeout=10)
            if response.status_code != 200:
                return None
                
            image = Image.open(BytesIO(response.content))
            return image.convert("RGB") if image else None
            
        except Exception as e:
            print(f"[ERROR] loading image: {e}")
            return None

# 12. Huấn luyện mô hình

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def evaluate_model(model, val_dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            encoder_outputs = model.encoder(pixel_values=pixel_values)
            outputs = model.decoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                encoder_hidden_states=encoder_outputs.last_hidden_state,
                labels=input_ids
            )
            loss = outputs.loss
            total_loss += loss.item()
    return total_loss / len(val_dataloader)

def train_model(model, train_dataloader, val_dataloader, tokenizer, feature_extractor, lr=5e-5, epochs=1, epoch_count=1, model_name='BartPho_ViT_GPT2_LoRA_ICG', data_part="Part_1", df_log=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Optimizer chỉ update các tham số có requires_grad=True (LoRA adapter)
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    # Thống kê tham số trainable
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())

    run_name = model_name + "_" + data_part

    with mlflow.start_run(run_name=run_name):
        if df_log is not None:
            dataset = mlflow.data.from_pandas(df_log, targets="caption")
            mlflow.log_input(dataset, context="training")

        # Logging các siêu tham số
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("epoch_count", epoch_count)
        mlflow.log_param("data_part", data_part)
        mlflow.log_param("learning_rate", lr)
        mlflow.log_param("train_data_size", len(train_dataloader.dataset))
        mlflow.log_param("val_data_size", len(val_dataloader.dataset))
        mlflow.log_param("trainable_params", trainable_params)
        mlflow.log_param("total_params", total_params)
        mlflow.log_param("trainable_ratio", round(trainable_params / total_params, 4))
        mlflow.log_param("model_name", model_name)

        model.train()
        for epoch in range(epochs):
            total_loss = 0
            for batch in train_dataloader:
                pixel_values = batch["pixel_values"].to(device)
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)

                encoder_outputs = model.encoder(pixel_values=pixel_values)
                outputs = model.decoder(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    encoder_hidden_states=encoder_outputs.last_hidden_state,
                    labels=input_ids
                )
                loss = outputs.loss
                total_loss += loss.item()

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            avg_loss = total_loss / len(train_dataloader)
            val_loss = evaluate_model(model, val_dataloader, device)

            # Log loss theo epoch
            mlflow.log_metric("train_loss", avg_loss, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)
            print(f"[Epoch {epoch+1}/{epochs}] Train Loss: {avg_loss:.4f} | Val Loss: {val_loss:.4f}")

            # Lưu mô hình sau mỗi epoch
            epoch_save_path = f"/kaggle/working/{model_name}_epoch_{epoch+1}"
            model.save_pretrained(epoch_save_path)
            tokenizer.save_pretrained(os.path.join(epoch_save_path, "tokenizer"))
            feature_extractor.save_pretrained(os.path.join(epoch_save_path, "feature_extractor"))
            
            # Log toàn bộ thư mục như một artifact
            mlflow.log_artifacts(epoch_save_path, artifact_path=f"epoch_{epoch+1}")

        # Lưu model cuối cùng riêng biệt
        final_model_path = f"/kaggle/working/{model_name}_final"
        model.save_pretrained(final_model_path)
        tokenizer.save_pretrained(os.path.join(final_model_path, "tokenizer"))
        feature_extractor.save_pretrained(os.path.join(final_model_path, "feature_extractor"))
        mlflow.log_artifacts(final_model_path, artifact_path="final_model")

        # Log model dưới dạng pyfunc (nếu cần)
        # Đảm bảo bạn đã định nghĩa class ImageCaptionModel trước

        # Infer signature
        input_example = pd.DataFrame({"url": ["http://example.com/test.jpg"]})
        output_example = pd.DataFrame({"caption": ["Mô tả ảnh"]})
        
        # 2. Tạo model signature
        signature = infer_signature(
            input_example,
            output_example,
            params={"input_types": "string", "output_types": "string"}  # Chỉ định kiểu dữ liệu
        )
        mlflow.pyfunc.log_model(
            artifact_path="model",
            python_model=LoRACaptionWrapper(),
            artifacts={"model_dir": final_model_path},  # Đường dẫn đến thư mục chứa model, tokenizer và feature_extractor
            registered_model_name=model_name,
            signature=signature,
            # input_example=input_example,
            pip_requirements=pip_reqs
        )

In [ ]:
# Tạo k Subset cho train, val
train_subset = Subset(train_dataset, range(train_df.shape[0]))  # Lấy train_df.shape[0] dữ liệu đầu tiên từ train_dataset
val_subset = Subset(val_dataset, range(val_df.shape[0]))     # Lấy 4620 dữ liệu đầu tiên từ val_dataset

# Tạo DataLoader cho các subset
train_dataloader = DataLoader(train_subset, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)
val_dataloader = DataLoader(val_subset, batch_size=4, shuffle=False, collate_fn=custom_collate_fn)


# 13. Load model pretrain

In [ ]:
# Đường dẫn thư mục chứa các tệp mô hình
model_path = '/kaggle/input/phovit-gptcap/pytorch/default/10'

# Tải mô hình VisionEncoderDecoder
model = VisionEncoderDecoderModel.from_pretrained(model_path, ignore_mismatched_sizes=True)

model = model.to(device)

# 14. Huấn luyện

In [ ]:
# Huấn luyện mô hình
train_model(
    model,
    train_dataloader,
    val_dataloader,
    tokenizer,
    feature_extractor,
    lr=5e-5,
    epochs=5,
    epoch_count=15,
    model_name='BartPho_ViT_GPT2_LoRA_ICG',
    data_part="FULL",
    df_log=df
)